## 🤖 Criando um agente conversacional

Instalando as bibliotecas de LangChain:

In [ ]:
%pip install langchain==0.3.0
%pip install langchain-groq==0.2.0
%pip install langchain-community==0.3.0
%pip install youtube_transcript_api==0.6.2
%pip install pypdf==5.0.0

Importando bibliotecas LangChain e configurando chave de API do modelo Groq:

In [ ]:
import os
from langchain_groq import ChatGroq

api_key = 'YOUR_GROQ_API_KEY'
os.environ['GROQ_API_KEY'] = api_key
chat = ChatGroq(model='GROQ_MODEL')

Criando chain e função de resposta aplicando templates para o modelo de IA:

In [ ]:
from langchain.prompts import ChatPromptTemplate

def response_bot(messages, documents):
  system_prompt = '''Você é um assistente amigável chamado Asimo.
  Você utiliza as seguintes informações para formular as suas respostas:
  {documents}'''

  model_message = [('system', system_prompt)]
  model_message += messages

  template = ChatPromptTemplate.from_messages(model_message)

  chain = template | chat
  return chain.invoke({'documents': documents}).content

Criando funções que vão servir para obter informações necessárias para contextualização que nossa IA irá trabalhar:

In [ ]:
from langchain_community.document_loaders import WebBaseLoader, YoutubeLoader, PyPDFLoader

def load_website_content():
  url = input('🔗 Insira a URL do website: ')

  if ('http' not in url):
    print ('URL inválida. Por favor, insira uma URL válida.')
    load_website_content()

  loader = WebBaseLoader(url)
  pages_content = loader.load()

  all_content = ''
  for doc in pages_content:
    all_content += doc.page_content

  return all_content

def load_pdf_content():
  path = 'PDF_PATH'
  loader = PyPDFLoader(path)
  pages_content = loader.load()

  all_content = ''
  for doc in pages_content:
    all_content += doc.page_content
  
  return all_content

def load_youtube_content():
  url = input('🔗 Insira a URL de um vídeo (Youtube): ')

  if ('youtube' not in url):
    print ('URL inválida. Por favor, insira uma URL válida.')
    load_youtube_content()

  loader = YoutubeLoader.from_youtube_url(url, language=['pt'])
  pages_content = loader.load()

  all_content = ''
  for doc in pages_content:
    all_content += doc.page_content
  
  return all_content

Desenvolvendo seção para dar ao usuário o poder de escolher com quais documentos ele vai querer que o chatbot se baseia:

In [ ]:
username = input('Digite o seu nome: ')

init_text = f'''Olá, {username}! Vamos começar a conversar?

Escolha uma das opções abaixo:
1. Conversar com contexto de um site
2. Conversar com contexto de um PDF
3. Conversar com contexto de um vídeo de Youtube
x. Sair

{username}: '''

while True:
  user_choice = input(init_text)
  if user_choice == '1':
    document = load_website_content()
    break
  elif user_choice == '2':
    document = load_pdf_content()
    break
  elif user_choice == '3':
    document = load_youtube_content()
    break
  elif user_choice == 'x':
    break
  else:
    print('Valor incorretor. Tente novamente.') 

Bloco de código que itera mensagens do usuário e do bot para definir contextualização da conversa:

In [ ]:
messages = []
while True:
  question = input(f'{username}: ')
  if question.lower() == 'x':
    break
  messages.append(('user', question))
  response = response_bot(messages, document)
  messages.append(('assistant', response))
  print(f'Bot: {response}')

print('Espero conversar com você novamente!\nMuito obrigado por usar o chat!\n')